In [33]:
import re
import json
import os
import logging

In [59]:
def source_table(view_path):
    """
    This function extracts the source view/table from the `sql_table_name` parameter, or from the `extends` parameter.

    :param view_path: the path of the view file

    :type view_path: string
    
    :return: json payload of view name and the source table.
    """

    with open(view_path, 'r') as f:    
        view = json.load(f)

    view_source = {'view_name': view['view_name']}
    line_dict = {}
    
    for line in view['view']:
        line_list = list(filter(None, line.split(' ')))
        if len(line_list) > 0 and (bool(re.search('extends', line_list[0])) \
                                   or bool(re.search('sql_table_name', line_list[0]))):
            line_dict[line_list[0]] = line_list[1]
        elif len(line_list) > 0 and bool(re.search('derived_table', line_list[0])):
            view_source['view_type'] = 'derived_table'

    override_key_list = [bool(re.search('sql_table_name', x)) for x in line_dict.keys()]

    if len(override_key_list) >= 1 and sum(override_key_list) >= 1:
        view_source['source_table'] = line_dict['sql_table_name:']
        view_source['view_type'] = 'sql_table_name'
    elif len(override_key_list) == 1 and sum(override_key_list) == 0:
        view_source['source_table'] = line_dict['extends:'][1:-1]
        view_source['view_type'] = 'extension'

    if view_source['view_type'] == 'derived_table':
        derived_list = get_derived_table(view['view'])
        view_source['derived_table'] = derived_list
    
    return view_source

In [41]:
dir_path = '..'

In [61]:
for view in next(os.walk(f'{dir_path}/../views'))[2]:    
    print(f'Processing view file {view}') 
    print(f'Starting to parse view {view}...')
    
    view_path = f'{dir_path}/../views/{view}'
    result = source_table(view_path)
    result_json = json.dumps(result)
    


#         f = open(f'{dir_path}/../maps/view-{view_folder}-{view}', "w")
#         f.write(result_json)
#         f.close()

Processing view file store_ddc_login.json
Starting to parse view store_ddc_login.json...
{"view_name": "store_ddc_login", "source_table": "store.ddc_login", "view_type": "sql_table_name"}
Processing view file pinata_settings_changed.json
Starting to parse view pinata_settings_changed.json...
{"view_name": "pinata_settings_changed", "source_table": "beta_docker_for_desktops_app.action_settings_changed", "view_type": "sql_table_name"}
Processing view file finance_monthly_renewal_retention.json
Starting to parse view finance_monthly_renewal_retention.json...
{"view_name": "finance_monthly_renewal_retention", "source_table": "finance.monthly_renewal_retention", "view_type": "sql_table_name"}
Processing view file sf__leads_and_contacts_pdt.json
Starting to parse view sf__leads_and_contacts_pdt.json...
{"view_name": "sf__leads_and_contacts_pdt", "view_type": "derived_table", "derived_table": [["derived_table:", "{"], ["datagroup_trigger:", "snowflake_sfdc_lead_and_contact_data"], ["sql:"], [

{"view_name": "finance_customers_arr_segmentation", "source_table": "finance.customers_arr_segmentation", "view_type": "sql_table_name"}
Processing view file reghub_dockeruser.json
Starting to parse view reghub_dockeruser.json...
{"view_name": "reghub_dockeruser", "source_table": "public.reghub_dockeruser", "view_type": "sql_table_name"}
Processing view file ucp_sessions_user_actions.json
Starting to parse view ucp_sessions_user_actions.json...
{"view_name": "ucp_sessions_user_actions", "view_type": "derived_table", "derived_table": [["derived_table:", "{"], ["sql_trigger_value:", "select", "count(*)", "from", "${ucp_mapped_user_actions.SQL_TABLE_NAME}", ";;"], ["sortkeys:", "[\"session_id\"]"], ["distribution:", "\"ucp_user_id\""], ["sql:", "select", "row_number()", "over(partition", "by", "ucp_user_id", "order", "by", "received_at)", "||", "'", "-", "'", "||", "ucp_user_id", "as", "session_id"], [",", "ucp_user_id"], [",", "received_at", "as", "session_start_at"], [",", "row_number()

{"view_name": "sf__leads_and_contacts", "source_table": "${sf__leads_and_contacts_pdt.SQL_TABLE_NAME}", "view_type": "sql_table_name"}
Processing view file store_search.json
Starting to parse view store_search.json...
{"view_name": "store_search", "source_table": "store.search", "view_type": "sql_table_name"}
Processing view file sf__opportunity_product_active_ids.json
Starting to parse view sf__opportunity_product_active_ids.json...
{"view_name": "sf__opportunity_product_active_ids", "source_table": "salesforce.opportunity_product_active_ids", "view_type": "sql_table_name"}
Processing view file stripe_customers.json
Starting to parse view stripe_customers.json...
{"view_name": "stripe_customers", "source_table": "stripe.customers", "view_type": "sql_table_name"}
Processing view file namely_headcount_historical.json
Starting to parse view namely_headcount_historical.json...
{"view_name": "namely_headcount_historical", "view_type": "derived_table", "derived_table": [["#", "View", "to", 

In [62]:
def get_true_source(dir_path, view_payload, explore, connection_map, view_map):
    
    if isinstance(view_payload, dict):

        if view_payload['view_type'] == 'extension': 

            new_view_name = look_up_target_view(view_payload['source_table'], view_map=view_map)
            with open(f'{dir_path}/../maps/{new_view_name}', 'r') as f:
                new_view_payload = json.load(f)
            return get_true_source(dir_path, new_view_payload, explore, connection_map, view_map)
            
        elif view_payload['view_type'] == 'sql_table_name': 

            source = view_payload['source_table']

            if is_true_source(source):
                true_source = view_payload['source_table']

            else: 
                if len(source.split('.')) == 3:
                    true_source = source
                
                else:
                    provider, database = get_conn_db(explore=explore, connection_map=connection_map)
                    if len(source.split('.')) == 2:
                        true_source = f'{provider}.{source}'
                    elif len(source.split('.')) == 1:
                        true_source = f'{provider}.{database}.{source}'

        elif view_payload['view_type'] == 'derived_table':
            true_source = 'derived table'

    else:
        true_source = view_payload

    return view_payload['view_name'], true_source


In [63]:
dir_path

'..'

In [ ]:
view_content = dict()
view_map = dict()
# for view in next(os.walk(f'{dir_path}/../maps'))[2]:
#     if view.startswith('view'):
#         logging.info(f'Getting source tables for view {view}...')
view_map[view.split('-')[1].split('.')[0]] = view
with open(f'{dir_path}/../maps/{view}','r') as f:
    payload = json.load(f)
    view_content[payload['view_name']] = payload


In [ ]:
source_payload = dict()
for view_name,view_payload in view_content.items():
    logging.info(f"Processing View source {view_payload['view_name']}...")
    if view_name in explore['explore_joins']:
        base_view_name, source_table = get_true_source(f'{dir_path}', view_payload, explore=explore, connection_map=connection_map, view_map=view_map)
                 